In [6]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import glob
import time 
import numpy as np
from itertools import combinations 
from mwc_functions import *
from mwc_class import getMVWs
from optimization_functions import *
from power_indice_functions import * 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
start_time=time.time()
krohn_data = getMVWs('krohn.csv', name='krohn_example', save_results=True,verify_mwcs=True,encoding='UTF-8',delimiter=',')
prelims = krohn_data.preliminaries()
country_MIW = krohn_data.minimal_voting_weights_pipeline()
end_time=time.time()
duration_Verify=end_time-start_time

print(duration_Verify)

1.1327481269836426


In [4]:
krohn_data.alternative_weights

{'2001': [array([15, 13, 10,  8,  6,  4,  4,  2,  1]),
  array([15, 13, 10,  8,  6,  4,  4,  1,  2])],
 '2002': [array([17, 15, 11,  9,  7,  5,  4,  2,  1]),
  array([17, 15, 11,  9,  7,  5,  4,  1,  2])],
 '2003': [array([13,  7,  6,  6,  4,  4,  4,  3,  2]),
  array([13,  7,  6,  6,  4,  4,  4,  2,  3])],
 '2004': [array([15, 13, 10,  8,  6,  4,  4,  2,  1]),
  array([15, 13, 10,  8,  6,  4,  4,  1,  2])],
 '2005': [array([17, 15, 11,  9,  7,  5,  4,  2,  1]),
  array([17, 15, 11,  9,  7,  5,  4,  1,  2])],
 '2006': [array([13,  7,  6,  6,  4,  4,  4,  3,  2]),
  array([13,  7,  6,  6,  4,  4,  4,  2,  3])]}

In [69]:
krohn_data.power_indices_pipeline()

'Power Indices successfully saved'

# Create Functions that can calculate Banzhaf Index 

first get a list of mwc´s and parties for a year:

In [29]:
min_win_coals_year=minimal_winning_coalitions_for_a_year(krohn_data.minimal_winning_coalitions,'2001')
#relevant_parties_for_a_year=relevant_parties_for_a_year(krohn_data.minimal_winning_coalitions,'2001')

In [30]:
min_win_coals_year

['A+B+C',
 'A+B+D',
 'A+B+E',
 'A+B+F',
 'A+B+G',
 'A+C+D',
 'A+C+E+F',
 'A+C+E+G',
 'A+C+E+H',
 'A+C+E+I',
 'A+C+F+G',
 'A+D+E+F',
 'A+D+E+G',
 'B+C+D+E',
 'B+C+D+F',
 'B+C+D+G',
 'B+C+D+H',
 'B+C+D+I',
 'B+C+E+F',
 'B+C+E+G',
 'A+C+F+H+I',
 'A+C+G+H+I',
 'A+D+E+H+I',
 'A+D+F+G+H',
 'A+D+F+G+I',
 'B+C+E+H+I',
 'B+C+F+G+H',
 'B+C+F+G+I',
 'B+D+E+F+G',
 'B+D+E+F+H',
 'B+D+E+F+I',
 'B+D+E+G+H',
 'B+D+E+G+I',
 'C+D+E+F+G',
 'A+E+F+G+H+I',
 'B+D+F+G+H+I']

create a function that can create a list of any 2 mwc unions 

In [4]:
mwc_sets = [set(mwc.split('+')) for mwc in min_win_coals_year]

In [6]:

def union_of_two_mwcs_as_sets(min_win_coals_year):
    mwc_sets = [set(mwc.split('+')) for mwc in min_win_coals_year]
    combined_coalitions_sets = []
    for mwc1, mwc2 in combinations(mwc_sets, 2):
        combined_set = mwc1.union(mwc2) 
        combined_coalitions_sets.append(combined_set)
    return combined_coalitions_sets


In [7]:
test_set_2 = union_of_two_mwcs_as_sets(min_win_coals_year)

extend this to n-mwc-unions:

In [ ]:
def union_of_n_mwcs_as_sets(min_win_coals_year, n):
    mwc_sets = [set(mwc.split('+')) for mwc in min_win_coals_year]
    combined_coalitions_sets = []
    for mwc_combination in combinations(mwc_sets, n):
        combined_set = set().union(*mwc_combination)
        combined_coalitions_sets.append(combined_set)
    return combined_coalitions_sets

In [ ]:
test_set_4=union_of_n_mwcs_as_sets(min_win_coals_year,4)

now the opus magnum of this code: 

a function which creates all n-unions for n= number of mwcs \\

for any party consider only sets in which that party is a member \\

for those sets calculate $2^{n-\#\{\bigcup(\dots)\}}$ \\

sum over all those and add or substract them from BS_i for all n \\

this generally follows the steps of the example in "Power indices and minimal winning coalitions",Werner Kirsch, Jessica Langner,"Social Choice and Welfare, Vol. 34, No. 1 (January 2010), pp. 33-46 (14 pages)"
https://www.jstor.org/stable/41108037

In [ ]:
def BS_w(min_win_coals_year,relevant_parties): # to stay in notation of the paper, indicates the Banzhaf score of a voter w -> in our case score of a party 
    steps=len(min_win_coals_year)
    BS={}
    coalition_sets = {}
    for i in range(1,steps+1): 
       coalition_sets[i]= union_of_n_mwcs_as_sets(min_win_coals_year, i)
    for party_i in relevant_parties:
        BS_i = 0
        for i in range(1,steps+1): #We need a step for any value in steps (1,2,...,len(min_win_coals_year))
            coalition_sets_with_i = [coalition_set for coalition_set in coalition_sets[i] if party_i in coalition_set] #in words: list every coalition set from coalition_sets[i] that includes party i
            change_of_BS_i = sum(2**(steps - len(coalition_set)) for coalition_set in coalition_sets_with_i) #in words: for every set in coalition_sets_with_i calculate $2^{n-#U(...)}$ and add them all togehter
            if i % 2 == 1:  #  if i odd: add change_of_BS_i
                BS_i += change_of_BS_i
            else:  # if i even: substract change_of_BS_i
                BS_i -= change_of_BS_i
        BS[party_i]=BS_i      


In [4]:
banzaf_dict=BS_w(min_win_coals_year,relevant_parties_for_a_year)

KeyboardInterrupt: 

this approach is just much to slow, it requires per step in length listing all nCr (length over n) possibilities, which for the case of 36 over 18 is around 9e9 and calculations for every one of them

# try classical approach

In [28]:
germany.optimal_seats

{'1994': {'CDU - Christian Democratic Union (Christlich Demokratische Union , CDU)': 2.0,
  'CSU - Christian Social Union  (Christlich Soziale Union, CSU)': 1.0,
  'FDP - Free Democrats  (Freie Demokratische Partei, FDP)': 1.0,
  'G - Alliance 90-Greens (Bundnis 90-Die Grunen, G)': 1.0,
  'PDS - Party of Democratic Socialism (Partei des Demokratischen Sozialismus, PDS)': 0.0,
  'SPD - Social Democrats (Sozialdemokratische Partei Deutschlands, SPD)': 2.0},
 '1998': {'CDU - Christian Democratic Union (Christlich Demokratische Union , CDU)': 1.0,
  'CSU - Christian Social Union  (Christlich Soziale Union, CSU)': 1.0,
  'FDP - Free Democrats  (Freie Demokratische Partei, FDP)': 1.0,
  'G - Alliance 90-Greens (Bundnis 90-Die Grunen, G)': 1.0,
  'PDS - Party of Democratic Socialism (Partei des Demokratischen Sozialismus, PDS)': 0.0,
  'SPD - Social Democrats (Sozialdemokratische Partei Deutschlands, SPD)': 3.0},
 '2002': {'CDU - Christian Democratic Union (Christlich Demokratische Union , CD

In [5]:
weights_dict= grab_relevant_weights(krohn_data.optimal_seats,'2001')
weights= weight_dict_to_array(weights_dict)
min_win_coals_year=minimal_winning_coalitions_for_a_year(krohn_data.minimal_winning_coalitions,'2001')
min_len=mincardinality(min_win_coals_year)

In [17]:
weights_dict

{'A': (15, 15),
 'B': (13, 13),
 'C': (10, 10),
 'D': (8, 8),
 'E': (6, 6),
 'F': (4, 4),
 'G': (4, 4),
 'H': (2, 1),
 'I': (1, 2)}

In [24]:
sum(weights_dict.get('H'))/len(weights_dict.get('H'))

1.5

In [13]:
pb_s,ss_list = pb_and_ss_i(weights,min_len)
(pb_s,ss_list)

([0.515625,
  0.421875,
  0.3125,
  0.234375,
  0.1875,
  0.125,
  0.125,
  0.046875,
  0.046875],
 [0.26349206349206344,
  0.21587301587301583,
  0.15158730158730158,
  0.11587301587301586,
  0.08968253968253968,
  0.06111111111111111,
  0.06111111111111111,
  0.020634920634920634,
  0.020634920634920634])

In [33]:
msr_i=msr_index_i(weights_dict)
(msr_i)

array([0.23809524, 0.20634921, 0.15873016, 0.12698413, 0.0952381 ,
       0.06349206, 0.06349206, 0.02380952, 0.02380952])

In [40]:
parties=relevant_parties_for_a_year(krohn_data.minimal_winning_coalitions,'2001')
(parties)

{'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'}

In [46]:
df = combine_names_and_indices(weights_dict,pb_s,ss_list,msr_i)
(df)

,Party,Penrose-Banzhaf,Shapely-Shubik,Minimal-Sum
0,A,0.515625,0.263492,0.238095
1,B,0.421875,0.215873,0.206349
2,C,0.312500,0.151587,0.158730
3,D,0.234375,0.115873,0.126984
4,E,0.187500,0.089683,0.095238
5,F,0.125000,0.061111,0.063492
6,G,0.125000,0.061111,0.063492
7,H,0.046875,0.020635,0.023810
8,I,0.046875,0.020635,0.023810


In [49]:
df = power_indices_year(krohn_data.optimal_seats,krohn_data.minimal_winning_coalitions,'2001')

In [52]:
krohn_data.minimal_winning_coalitions

{('2001', 'A+B+C'): 1,
 ('2001', 'A+B+D'): 1,
 ('2001', 'A+B+E'): 1,
 ('2001', 'A+B+F'): 1,
 ('2001', 'A+B+G'): 1,
 ('2001', 'A+C+D'): 1,
 ('2001', 'A+C+E+F'): 1,
 ('2001', 'A+C+E+G'): 1,
 ('2001', 'A+C+E+H'): 1,
 ('2001', 'A+C+E+I'): 1,
 ('2001', 'A+C+F+G'): 1,
 ('2001', 'A+D+E+F'): 1,
 ('2001', 'A+D+E+G'): 1,
 ('2001', 'B+C+D+E'): 1,
 ('2001', 'B+C+D+F'): 1,
 ('2001', 'B+C+D+G'): 1,
 ('2001', 'B+C+D+H'): 1,
 ('2001', 'B+C+D+I'): 1,
 ('2001', 'B+C+E+F'): 1,
 ('2001', 'B+C+E+G'): 1,
 ('2001', 'A+C+F+H+I'): 1,
 ('2001', 'A+C+G+H+I'): 1,
 ('2001', 'A+D+E+H+I'): 1,
 ('2001', 'A+D+F+G+H'): 1,
 ('2001', 'A+D+F+G+I'): 1,
 ('2001', 'B+C+E+H+I'): 1,
 ('2001', 'B+C+F+G+H'): 1,
 ('2001', 'B+C+F+G+I'): 1,
 ('2001', 'B+D+E+F+G'): 1,
 ('2001', 'B+D+E+F+H'): 1,
 ('2001', 'B+D+E+F+I'): 1,
 ('2001', 'B+D+E+G+H'): 1,
 ('2001', 'B+D+E+G+I'): 1,
 ('2001', 'C+D+E+F+G'): 1,
 ('2001', 'A+E+F+G+H+I'): 1,
 ('2001', 'B+D+F+G+H+I'): 1,
 ('2002', 'A+B+C'): 1,
 ('2002', 'A+B+D'): 1,
 ('2002', 'A+B+E'): 1,
 ('2002

In [54]:
test = get_power_indices(krohn_data.optimal_seats,krohn_data.minimal_winning_coalitions)

In [55]:
test

{'2001':   Party  Penrose-Banzhaf  Shapely-Shubik  Minimal-Sum
 0     A         0.515625        0.263492     0.238095
 1     B         0.421875        0.215873     0.206349
 2     C         0.312500        0.151587     0.158730
 3     D         0.234375        0.115873     0.126984
 4     E         0.187500        0.089683     0.095238
 5     F         0.125000        0.061111     0.063492
 6     G         0.125000        0.061111     0.063492
 7     H         0.046875        0.020635     0.023810
 8     I         0.046875        0.020635     0.023810,
 '2002':   Party  Penrose-Banzhaf  Shapely-Shubik  Minimal-Sum
 0     A         0.507812        0.260317     0.239437
 1     B         0.429688        0.219841     0.211268
 2     C         0.304688        0.148413     0.154930
 3     D         0.242188        0.119841     0.126761
 4     E         0.195312        0.093651     0.098592
 5     F         0.132812        0.065079     0.070423
 6     G         0.117188        0.057937     0.

In [5]:
country_data = getMVWs(f'israel_99_modified.csv', name='israel_99_modified', save_results=True,verify_mwcs=True,find_errors=True)
prelims = country_data.preliminaries()
country_MIW = country_data.minimal_voting_weights_pipeline()
country_power_indices = country_data.power_indices_pipeline()

An error occurred: UTF-16 stream does not start with BOM
year:-Mar-06, started at 1707749137.1383548 seconds
year:-Mar-06, ended at 1707749137.178829 seconds, took 0.04047417640686035 seconds
year:-Mar-15, started at 1707749137.178829 seconds
year:-Mar-15, ended at 1707749137.1869242 seconds, took 0.008095264434814453 seconds
year:-Mar-20, started at 1707749137.1869242 seconds
year:-Mar-20, ended at 1707749137.1890118 seconds, took 0.0020875930786132812 seconds
year:-May-96, started at 1707749137.1890118 seconds
year:-May-96, ended at 1707749137.199445 seconds, took 0.010433197021484375 seconds
year:-May-99, started at 1707749137.199445 seconds
year:-May-99, ended at 1707749138.4182339 seconds, took 1.2187888622283936 seconds
year:1992-06, started at 1707749138.4182339 seconds
year:1992-06, ended at 1707749138.4198005 seconds, took 0.0015666484832763672 seconds
year:2003-01, started at 1707749138.4198005 seconds
year:2003-01, ended at 1707749138.4603448 seconds, took 0.0405442714691162

KeyboardInterrupt: 